In [ ]:
import os

os.chdir("/home/jovyan/ladec-2")

In [1]:
%pip install -r requirements.txt

ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements.txt'
Note: you may need to restart the kernel to use updated packages.


In [6]:
%pip install polars

  Obtaining dependency information for polars from https://files.pythonhosted.org/packages/e5/25/bf5d43dcb538bf6573b15f3d5995a52be61b8fbce0cd737e72c4d25eef88/polars-1.9.0-cp38-abi3-manylinux_2_24_aarch64.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.8/29.8 MB 7.6 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


In [235]:
import polars as pl
import emoji

from src.solutions import Baseline

Optimization of solution's exec time will be done using the polars framework, since it lets us define a schema and use lazy frames for execution optimization

In [32]:
dataset = "./data/farmers-protest-tweets-2021-2-4.json"

# Q1


In [125]:
%%time
Baseline(dataset).q1()

CPU times: user 2.63 s, sys: 59.8 ms, total: 2.69 s
Wall time: 2.69 s


[(datetime.date(2021, 2, 12), 'RanbirS00614606'),
 (datetime.date(2021, 2, 13), 'MaanDee08215437'),
 (datetime.date(2021, 2, 17), 'RaaJVinderkaur'),
 (datetime.date(2021, 2, 16), 'jot__b'),
 (datetime.date(2021, 2, 14), 'rebelpacifist'),
 (datetime.date(2021, 2, 18), 'neetuanjle_nitu'),
 (datetime.date(2021, 2, 15), 'jot__b'),
 (datetime.date(2021, 2, 20), 'MangalJ23056160'),
 (datetime.date(2021, 2, 23), 'Surrypuria'),
 (datetime.date(2021, 2, 19), 'Preetm91')]

In [185]:
%%time
# pdf = pl.read_ndjson(dataset, ignore_errors = True)
# schema = pdf.schema
schema = {
    "date": pl.String(),
    "content": pl.String(),
    "user": pl.Struct({"username": pl.String()}),
    "mentionedUsers": pl.List(pl.Struct({"username": pl.String()}))
}
plf = pl.read_ndjson(
    dataset,
    schema=schema,
)


CPU times: user 821 ms, sys: 59.2 ms, total: 880 ms
Wall time: 883 ms


In [186]:
%%time
response = (
    plf.select(
        pl.col('date').str.to_datetime().dt.truncate('1d'),
        pl.col("user").struct.field("username"),
    )
    .group_by("date", "username")
    .agg(pl.len().alias("freq"))
    .with_columns(max_freq=pl.col("freq").max().over("date"))
    .group_by("date")
    .agg(
        pl.when(pl.col("freq") == pl.col("max_freq")).then("username").drop_nulls().first(),
        pl.col("freq").sum(),
    )
    .sort("freq", descending = True)
    .select("date", "username")
    .limit(10)
)
# response = query.collect()
list(zip(*response.to_dict(as_series = False).values()))


CPU times: user 100 ms, sys: 7.73 ms, total: 108 ms
Wall time: 73.7 ms


[(datetime.datetime(2021, 2, 12, 0, 0, tzinfo=zoneinfo.ZoneInfo(key='UTC')),
  'RanbirS00614606'),
 (datetime.datetime(2021, 2, 13, 0, 0, tzinfo=zoneinfo.ZoneInfo(key='UTC')),
  'MaanDee08215437'),
 (datetime.datetime(2021, 2, 17, 0, 0, tzinfo=zoneinfo.ZoneInfo(key='UTC')),
  'RaaJVinderkaur'),
 (datetime.datetime(2021, 2, 16, 0, 0, tzinfo=zoneinfo.ZoneInfo(key='UTC')),
  'jot__b'),
 (datetime.datetime(2021, 2, 14, 0, 0, tzinfo=zoneinfo.ZoneInfo(key='UTC')),
  'rebelpacifist'),
 (datetime.datetime(2021, 2, 18, 0, 0, tzinfo=zoneinfo.ZoneInfo(key='UTC')),
  'neetuanjle_nitu'),
 (datetime.datetime(2021, 2, 15, 0, 0, tzinfo=zoneinfo.ZoneInfo(key='UTC')),
  'jot__b'),
 (datetime.datetime(2021, 2, 20, 0, 0, tzinfo=zoneinfo.ZoneInfo(key='UTC')),
  'MangalJ23056160'),
 (datetime.datetime(2021, 2, 23, 0, 0, tzinfo=zoneinfo.ZoneInfo(key='UTC')),
  'Surrypuria'),
 (datetime.datetime(2021, 2, 19, 0, 0, tzinfo=zoneinfo.ZoneInfo(key='UTC')),
  'Preetm91')]

# Q2

In [155]:
%%time
Baseline(dataset).q2()

CPU times: user 10.8 s, sys: 183 ms, total: 11 s
Wall time: 10.5 s


[('🙏', 5049),
 ('😂', 3072),
 ('🚜', 2972),
 ('🌾', 2182),
 ('🇮🇳', 2086),
 ('🤣', 1668),
 ('✊', 1651),
 ('❤️', 1382),
 ('🙏🏻', 1317),
 ('💚', 1040)]

In [277]:
%%time
(
    plf.select("content").with_columns(
        emojis = pl.col("content").map_elements(
            lambda x: [token.chars for token in emoji.analyze(x)],
            return_dtype=pl.List(pl.String())
        )
    )
    .filter(pl.col("emojis").list.len() > 0)
    .select(pl.col("emojis").list.explode())
    .group_by("emojis").len()
    .sort("len", descending = True)
    .limit(10)
)
list(zip(*response.to_dict(as_series = False).values()))


CPU times: user 8.5 s, sys: 0 ns, total: 8.5 s
Wall time: 8.49 s


[('narendramodi', 2265),
 ('Kisanektamorcha', 1840),
 ('RakeshTikaitBKU', 1644),
 ('PMOIndia', 1427),
 ('RahulGandhi', 1146),
 ('GretaThunberg', 1048),
 ('RaviSinghKA', 1019),
 ('rihanna', 986),
 ('UNHumanRights', 962),
 ('meenaharris', 926)]

# Q3

In [182]:
%%time
Baseline(dataset).q3()

CPU times: user 2.7 s, sys: 115 ms, total: 2.82 s
Wall time: 2.81 s


[('narendramodi', 2265),
 ('Kisanektamorcha', 1840),
 ('RakeshTikaitBKU', 1644),
 ('PMOIndia', 1427),
 ('RahulGandhi', 1146),
 ('GretaThunberg', 1048),
 ('RaviSinghKA', 1019),
 ('rihanna', 986),
 ('UNHumanRights', 962),
 ('meenaharris', 926)]

In [190]:
%%time
response = (
    plf.filter(pl.col("mentionedUsers").is_not_null())
    .select(pl.col("mentionedUsers").explode().struct.field("username"))
    .group_by("username")
    .len()
    .sort("len", descending = True)
    .limit(10)
)
# response = query.collect()
list(zip(*response.to_dict(as_series = False).values()))


CPU times: user 28.4 ms, sys: 14.7 ms, total: 43.1 ms
Wall time: 23.5 ms


[('narendramodi', 2265),
 ('Kisanektamorcha', 1840),
 ('RakeshTikaitBKU', 1644),
 ('PMOIndia', 1427),
 ('RahulGandhi', 1146),
 ('GretaThunberg', 1048),
 ('RaviSinghKA', 1019),
 ('rihanna', 986),
 ('UNHumanRights', 962),
 ('meenaharris', 926)]